## TEST 1

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import preprocessing
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
## reading pandas
os.chdir('/Users/andreasabia/documents/ESCP/Machine Learning With Python')
df = pd.read_csv('titanic_data_ESCP.csv')

In [3]:
df.columns

Index(['PassengerId', 'Survived', 'Name', 'Sex', 'Age', 'Siblings - Spouse',
       'Parents - Children', 'Ticket', 'Ticket class', 'Fare', 'Cabin',
       'Port of embarkation'],
      dtype='object')

#### Q1

In [4]:
# how many passenger were there
df['PassengerId'].count()

1309

In [5]:
#how many passengers survived
df[df['Survived']==1].count()[1]

342

##### Q2

In [6]:
#How many females survived
df[(df['Survived']==1) & (df['Sex']=='female')].count()[1]

233

In [7]:
#how many males died
df[(df['Survived']==0) & (df['Sex']=='male')].count()[1]

468

##### Q3

In [8]:
#how many children were there on the boat
df[df['Age']<17].count()[4]

134

##### Q4

In [9]:
#how many kids died?
df[(df['Survived']==0) & (df['Age']<17)].count()[1]

45

##### Q5

In [10]:
#How many families
df[df['Parents - Children']!=0].count()[0]

307

In [16]:
df[(df['Siblings - Spouse']!=0) | (df['Parents - Children']!=0)].count()[0]

519

##### Q6

In [11]:
#male to female ratio
male=df[df['Sex']=='male'].count()[0]
female=df[df['Sex']=='female'].count()[0]

sex_ratio=female/male
print(sex_ratio)

0.5527876631079478


##### Q7

correlations

In [29]:
# Creating Children column to evaluate if children or not
df['Children']=1

In [30]:
for i in range(len(df.Age)):
    if df.Age[i]<17:
       df.Children[i]=0

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
#Normalizing columns
x = df[['Siblings - Spouse']].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['Siblings - Spouse']=x_scaled

In [32]:
x = df[['Parents - Children']].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['Parents - Children']=x_scaled

In [33]:
x = df[['Age']].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['Age']=x_scaled

In [34]:
x = df[['Fare']].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['Fare']=x_scaled

In [35]:
df = pd.get_dummies(df, columns=['Sex','Port of embarkation','Ticket class'])

In [36]:
df=df[[c for c in df if c not in ['Survived']] 
       + ['Survived']]

In [37]:
#Checking Data
for i in df.columns:
    print ("---- %s ---" % i)
    print (df[i].value_counts())

---- PassengerId ---
1309    1
449     1
431     1
432     1
433     1
       ..
874     1
875     1
876     1
877     1
1       1
Name: PassengerId, Length: 1309, dtype: int64
---- Name ---
Connolly, Miss. Kate                           2
Kelly, Mr. James                               2
Phillips, Miss. Alice Frances Louisa           1
McGough, Mr. James Robert                      1
Cacic, Miss. Manda                             1
                                              ..
Williams, Mr. Charles Duane                    1
Navratil, Master. Michel M                     1
Saether, Mr. Simon Sivertsen                   1
Bonnell, Miss. Caroline                        1
Renouf, Mrs. Peter Henry (Lillian Jefferys)    1
Name: Name, Length: 1307, dtype: int64
---- Age ---
0.298509    47
0.273456    43
0.260929    41
0.373669    40
0.223350    39
            ..
0.292246     1
0.329826     1
0.924840     1
0.304773     1
0.141927     1
Name: Age, Length: 98, dtype: int64
---- Siblings - S

In [38]:
corr=df.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

,PassengerId,Age,Siblings - Spouse,Parents - Children,Fare,Children,Sex_female,Sex_male,Port of embarkation_C,Port of embarkation_Q,Port of embarkation_S,Ticket class_1,Ticket class_2,Ticket class_3,Survived
PassengerId,1,0.029,-0.055,0.0089,0.031,0.047,-0.013,0.013,0.048,0.012,-0.048,0.026,0.023,-0.042,-0.005
Age,0.029,1,-0.24,-0.15,0.18,0.58,-0.064,0.064,0.086,-0.019,-0.076,0.39,-0.015,-0.34,-0.077
Siblings - Spouse,-0.055,-0.24,1,0.37,0.16,-0.33,0.11,-0.11,-0.048,-0.049,0.075,-0.034,-0.052,0.073,-0.035
Parents - Children,0.0089,-0.15,0.37,1,0.22,-0.31,0.21,-0.21,-0.0086,-0.1,0.073,-0.013,-0.01,0.02,0.082
Fare,0.031,0.18,0.16,0.22,1,0.0067,0.19,-0.19,0.29,-0.13,-0.17,0.6,-0.12,-0.42,0.26
Children,0.047,0.58,-0.33,-0.31,0.0067,1,-0.086,0.086,0.01,0.04,-0.035,0.13,-0.01,-0.1,-0.12
Sex_female,-0.013,-0.064,0.11,0.21,0.19,-0.086,1,-1,0.067,0.089,-0.12,0.11,0.029,-0.12,0.54
Sex_male,0.013,0.064,-0.11,-0.21,-0.19,0.086,-1,1,-0.067,-0.089,0.12,-0.11,-0.029,0.12,-0.54
Port of embarkation_C,0.048,0.086,-0.048,-0.0086,0.29,0.01,0.067,-0.067,1,-0.16,-0.78,0.33,-0.13,-0.17,0.17
Port of embarkation_Q,0.012,-0.019,-0.049,-0.1,-0.13,0.04,0.089,-0.089,-0.16,1,-0.49,-0.17,-0.12,0.24,0.0037


In [39]:
corr=df[df.columns[1:]].corr()['Survived'][:-1]
corr

Age                     -0.077221
Siblings - Spouse       -0.035322
Parents - Children       0.081629
Fare                     0.257307
Children                -0.121485
Sex_female               0.543351
Sex_male                -0.543351
Port of embarkation_C    0.168240
Port of embarkation_Q    0.003650
Port of embarkation_S   -0.155660
Ticket class_1           0.285904
Ticket class_2           0.093349
Ticket class_3          -0.322308
Name: Survived, dtype: float64

It appears as Sex has the greatest impact.
The ticket class also has a relevant impact (especially for first class and class 3).

### Predicting Survival

In [41]:
#Checking for missing values
features=df
total = features.isnull().sum().sort_values(ascending=False)
percent_1 = features.isnull().sum()/features.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(6)

,Total,%
Cabin,1014,77.5
Survived,418,31.9
Age,263,20.1
Fare,1,0.1
Ticket class_3,0,0.0
Name,0,0.0


In [42]:
#Deleting cabin because it has too many missing values
del features['Cabin']

In [43]:
missing_survived = features.ix[(features['Survived'].isnull())]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [44]:
del missing_survived['Survived']
missing_survived.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 891 to 1308
Data columns (total 16 columns):
PassengerId              418 non-null int64
Name                     418 non-null object
Age                      332 non-null float64
Siblings - Spouse        418 non-null float64
Parents - Children       418 non-null float64
Ticket                   418 non-null object
Fare                     417 non-null float64
Children                 418 non-null int64
Sex_female               418 non-null uint8
Sex_male                 418 non-null uint8
Port of embarkation_C    418 non-null uint8
Port of embarkation_Q    418 non-null uint8
Port of embarkation_S    418 non-null uint8
Ticket class_1           418 non-null uint8
Ticket class_2           418 non-null uint8
Ticket class_3           418 non-null uint8
dtypes: float64(4), int64(2), object(2), uint8(8)
memory usage: 32.7+ KB


In [45]:
new_df = features.ix[(features['Survived'] == 0) |(features['Survived'] == 1) ]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [46]:
new_df=new_df.fillna(new_df.mean())
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 17 columns):
PassengerId              891 non-null int64
Name                     891 non-null object
Age                      891 non-null float64
Siblings - Spouse        891 non-null float64
Parents - Children       891 non-null float64
Ticket                   891 non-null object
Fare                     891 non-null float64
Children                 891 non-null int64
Sex_female               891 non-null uint8
Sex_male                 891 non-null uint8
Port of embarkation_C    891 non-null uint8
Port of embarkation_Q    891 non-null uint8
Port of embarkation_S    891 non-null uint8
Ticket class_1           891 non-null uint8
Ticket class_2           891 non-null uint8
Ticket class_3           891 non-null uint8
Survived                 891 non-null float64
dtypes: float64(5), int64(2), object(2), uint8(8)
memory usage: 76.6+ KB


In [47]:
labels = np.array(new_df['Survived'])
features= new_df.drop(['Survived','Name','Ticket'], axis = 1)
feature_list = list(features.columns)
features = np.array(features)

In [48]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.33, stratify= labels ,random_state = 42)

In [49]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (596, 14)
Training Labels Shape: (596,)
Testing Features Shape: (295, 14)
Testing Labels Shape: (295,)


In [50]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [51]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.24 degrees.


In [52]:
predictions=np.round(predictions)
y_actu = pd.Series(test_labels, name='Actual')
y_pred = pd.Series(predictions, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion

Predicted,0.0,1.0
Actual,,
0.0,160,15
1.0,39,81


In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_actu, y_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.80      0.91      0.86       175
           1       0.84      0.68      0.75       120

    accuracy                           0.82       295
   macro avg       0.82      0.79      0.80       295
weighted avg       0.82      0.82      0.81       295



#### Extra code that revealed to be useless


In [ ]:
## Replace female with 0 and Male with 1
df['Sex']=df['Sex'].replace('male',1)
df['Sex']=df['Sex'].replace('female',0)

In [19]:
#One hot Encoding Port of Embarkation
x=df[['PassengerId','Port of embarkation']]
new = pd.concat([x[['PassengerId']],pd.get_dummies(df['Port of embarkation'])],1)
df=pd.merge(left=df, right=new, on='PassengerId')

In [20]:
#One hot Encoding Ticket Class
x=df[['PassengerId','Ticket class']]
new = pd.concat([x[['PassengerId']],pd.get_dummies(df['Ticket class'])],1)
df=pd.merge(left=df, right=new, on='PassengerId')

In [22]:
#Renaming Columns
x=list(df.columns)
x.index(1)
x[16:19]=['TF1','TF2','TF3']
df.columns=x